In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"cavidannuriyev","key":"0e7d9b903fbfb17026c86180e4709dfa"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sahideseker/spam-mail-classifier-dataset

Dataset URL: https://www.kaggle.com/datasets/sahideseker/spam-mail-classifier-dataset
License(s): CC-BY-SA-4.0


In [ ]:
!unzip '/content/spam-mail-classifier-dataset.zip'

In [ ]:
df = pd.read_csv('/content/spam_mail_classifier.csv')

In [ ]:
df

,email_text,label
0,Let's catch up sometime next week!,ham
1,Don't forget to submit your project by Friday.,ham
2,Win a free iPhone now!!! Click here.,spam
3,Can you send me the report when it's ready?,ham
4,Meeting has been rescheduled to next Monday.,ham
...,...,...
995,Don't forget to submit your project by Friday.,ham
996,Meeting has been rescheduled to next Monday.,ham
997,Let's catch up sometime next week!,ham
998,Win a free iPhone now!!! Click here.,spam


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   email_text  1000 non-null   object
 1   label       1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [ ]:
df.describe()

,email_text,label
count,1000,1000
unique,10,2
top,Let's catch up sometime next week!,ham
freq,129,583


In [ ]:
df_shuffled = df.sample(frac = 1)

In [ ]:
df_shuffled

,email_text,label
10,Don't forget to submit your project by Friday.,ham
344,"Hey, are we still on for lunch tomorrow?",ham
476,Can you send me the report when it's ready?,ham
899,Congratulations! You’ve been selected for a ca...,spam
380,"Hey, are we still on for lunch tomorrow?",ham
...,...,...
873,Congratulations! You’ve been selected for a ca...,spam
844,Can you send me the report when it's ready?,ham
12,Let's catch up sometime next week!,ham
498,Let's catch up sometime next week!,ham


In [ ]:
X = df_shuffled['email_text']
y = df_shuffled['label']

In [ ]:
y.value_counts()

,count
label,
ham,583
spam,417


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse_output = False)

y_train = onehot.fit_transform(y_train.values.reshape(-1, 1))
y_test = onehot.fit_transform(y_test.values.reshape(-1, 1))

In [ ]:
max_length = round(sum([len(i.split()) for i in X_train]) / (len(X_train)))

In [ ]:
max_length

7

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

max_vocab_length = 10000
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [ ]:
text_vectorizer.adapt(X_train)

In [ ]:
sample_sentence = 'Please verify your request'

In [ ]:
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 7), dtype=int64, numpy=array([[1, 1, 7, 1, 0, 0, 0]])>

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             embeddings_initializer = 'uniform',
                             input_length = max_length,
                             name='embedding')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


#Model

In [ ]:
inputs = layers.Input(shape=(1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, activation = 'relu', kernel_regularizer='l2')(x)
outputs = layers.Dense(len(df['label'].value_counts()), activation = 'sigmoid')(x)
model = tf.keras.Model(inputs, outputs, name='model_0')

In [ ]:
model.compile(loss='categorical_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
model_history = model.fit(X_train.to_numpy(),
                          y_train,
                          epochs = 5,
                          validation_data = (X_test.to_numpy(), y_test))

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.8844 - loss: 2.1065 - val_accuracy: 1.0000 - val_loss: 1.2185
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 1.0000 - loss: 0.9488 - val_accuracy: 1.0000 - val_loss: 0.4908
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 1.0000 - loss: 0.4131 - val_accuracy: 1.0000 - val_loss: 0.2251
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.1863 - val_accuracy: 1.0000 - val_loss: 0.0979
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 1.0000 - loss: 0.0805 - val_accuracy: 1.0000 - val_loss: 0.0407


In [ ]:
prediction_position = np.argmax(model.predict((tf.constant(['Iranian deputy FM set to visit Azerbaijan Baku']))))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [ ]:
prediction_position

np.int64(0)

In [ ]:
models_prediction = onehot.get_feature_names_out()[prediction_position][3:]
models_prediction

'ham'

In [ ]:
prediction_position_2 = np.argmax(model.predict((tf.constant(['Payment request details']))))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


In [ ]:
prediction_position_2

np.int64(1)

In [ ]:
models_prediction_2 = onehot.get_feature_names_out()[prediction_position_2][3:]
models_prediction_2

'spam'